## Imports

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

## loading data

In [13]:
def load_data_from_xml(filename):
    aspect_data = dict()
    sentences = list()
    tree = ET.parse(filename)
    root = tree.getroot()
    sentences = list()
    score= list()
    for row in root:
        sentences.append(row[2].text)
        temp = row[4].text.split('/')
        for aspect in temp:
            i = temp.index(aspect)
            if 'Level'+str(i) in aspect_data.keys():
                aspect_data['level'+str(i)].append(aspect)
            else:
                aspect_data['level'+str(i)] = [aspect]
    return aspect_data,sentences

In [14]:
aspect_data,sentences = load_data_from_xml('dataset/financial_posts_ABSA_train.xml')

In [15]:
sentences

['Slowly adding some $FIO here but gotta be careful. This will be one of biggest winners in 2012',
 '$TRX http://stks.co/1KkK Long setup. MACD cross.',
 'I am not optimistic about $amzn both fundementals and charts look like poopoo this quarter.',
 '$GRPN might be selling off ahead of $P earnings...',
 '$IACI http://stks.co/tJU Looks good on the weekly chart.',
 '$pcln back over "up" trendline from 10/4',
 'RT @robbieLOLZ $NFLX A close above here is looking good for the bulls. Will hold my positions to see where we close at. Keep buying here.',
 "Profit taking on $AAPL this morning?  That has to be the pressure on the stock.  I'm still bullish on $AAPL.",
 '$SKX turning. Coming from far could go far. Stock price implies you pay nothing for the business',
 '$MOS looking good here at $58.65. Calls are active in this month and weekly',
 'Sold out $TZA 45 $PUT (down $1), which were hedging my 45 $CALL. Letting the calls ride solo now.',
 '$YMI http://stks.co/Xyf Long setup. Closed above th

In [16]:
aspect_data

{'level0': ['Corporate'], 'level1': ['Strategy']}